## Import packages

In [1]:
%matplotlib widget

In [48]:
import os
import glob
import numpy  as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import ipywidgets as widgets
import wave
import yaml

### Choose file to inspect

In [3]:
directory='/Users/saroltagabulya/git/Orca/'
wavs=glob.glob('wav_files/*.wav')
file=[]

dropdown=widgets.Dropdown(
    options=wavs,
    description='Wav files:',
    disabled=False,
)

def dropdown_eventhandler(change):
    file.clear()
    file.append(change.new)
    print(change.new)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)

Dropdown(description='Wav files:', options=('wav_files/2015-11-10--15-25.wav', 'wav_files/2017-02-04--10-25-15…

wav_files/2017-02-04--10-25-15--00-05-25--C.wav


### Read metadata


In [4]:
# Set sample rate
try:
    wave_file=wave.open(file[0], "rb")
    sample_rate = wave_file.getframerate()
except:
    sample_rate=int(input('Please check sampling rate manually in the metadata file and set below in Hz! \n'))
    
# Set reference value
ref_value=int(input('Please check reference value in the files README and set below in uPa! \n'))

# Calibration value
cal_value=int(input('Please check calibration value in the files README and set below! \n what does the fullscale voltage correspond to in Pa? \n '))


# Set researcher
researcher= input('Please indicate the researcher who performs the crop by initials [SG, JR] \n ')

Please check sampling rate manually in the metadata file and set below in Hz! 
100000
Please check reference value in the files README and set below in uPa! 
1
Please check calibration value in the files README and set below! 
 what does the fullscale voltage correspond to in Pa? 
 1
Please indicate the researcher who performs the crop by initials [SG, JR] 
 SG


### Read in wav file

In [5]:
directory='/Users/saroltagabulya/git/Orca/call_segments/'

folder_name = directory + file[0][10:-4]
os.chdir(folder_name)
wavs=glob.glob('*.wav')
wavs

['2017-02-04--10-28-15--00-00-03--C.wav',
 '2017-02-04--10-28-45--00-00-03--C.wav',
 '2017-02-04--10-29-32--00-00-04--C.wav',
 '2017-02-04--10-30-18--00-00-04--C.wav',
 '2017-02-04--10-28-22--00-00-04--C.wav',
 '2017-02-04--10-28-08--00-00-04--C.wav',
 '2017-02-04--10-28-03--00-00-01--C.wav',
 '2017-02-04--10-29-58--00-00-03--C.wav',
 '2017-02-04--10-28-36--00-00-03--C.wav',
 '2017-02-04--10-29-52--00-00-03--C.wav']

### Choose specific wav

In [11]:
the_wav=wavs[-1]

the_wav

'2017-02-04--10-29-52--00-00-03--C.wav'

### Calculate spectogram

In [54]:
def open_wav(file, sample_rate):
    y, sr = librosa.load(file, sr=sample_rate, mono=False)
    return y, sr

def wav_to_spect_params(y, sr, file, nfft, hop_length, win_length): #, call_wav
    
    yml_file=file[:-3] + 'yml'


    with open(yml_file) as f:
        # The FullLoader parameter handles the conversion from YAML
        # scalar values to Python the dictionary format
        metadata = yaml.load(f, Loader=yaml.FullLoader)

    ref=metadata['reference_value']
    raw_unit=metadata['raw_signal_unit']
    start_utc=metadata['start_utc']
    duration_seconds=metadata['duration_seconds']

    
    if len(y)>0:
        # Convert to spectogram 
        
        Y = librosa.stft(y, n_fft=nfft, hop_length=hop_length, win_length=win_length) 
        Ydb = librosa.amplitude_to_db(abs(Y), ref=1)
        
        librosa.display.specshow(Ydb, cmap='seismic', sr=sr, x_axis='time', y_axis='fft', hop_length=32)
        
        cb=plt.colorbar()
        cb.set_label('Power [dB re to {} {}^2]'.format(ref, raw_unit), rotation=270, labelpad=+15)

        plt.xlabel('Time [s]')
        plt.ylabel('Frequency [kHz]'.format(ref, raw_unit))
        ticks, labels = plt.yticks()
        plt.yticks(ticks, [str(t / 1000) for t in ticks])
        plt.title('onset: {} (UTC) \nduration: {} seconds'.format(start_utc, duration_seconds))
        

        # Save spectograms
        plt.savefig(file[:-3] + 'png', bbox_inches='tight', dpi=600, facecolor='White')
        plt.close()
    
        return Ydb
    
    else: 
        return 'Error with wav'

In [55]:
#Close and previous graphs
plt.close()

In [65]:
nffts=[512, 2048, 8192]

hop_l=[32, 32, 32]

win_l=[512, 2048, 8192]

In [69]:
fig=plt.figure()

cols=3
rows=round(len(nffts)/3)


for p in range(len(nffts)):

    y, sr=open_wav(the_wav, sample_rate)

    Ydb=wav_to_spect_params(y, ref_value, the_wav, nffts[p], hop_l[p], win_l[p])

    #if type(Ydb)!= None: 

    #plt.figure(figsize=(round(np.shape(Ydb)[0]/10), round(np.shape(Ydb)[1]/10)))
    fig.add_subplot(cols, rows, p+1)

    # tell mpl_connect we want to pass a 'button_press_event' into onclick when the event is detected
    #plt.gcf().canvas.mpl_connect('key_press_event', onclick_for_w(w))

    librosa.display.specshow(Ydb, cmap='seismic', sr=sample_rate, x_axis='time', y_axis='fft')
    
    cb=plt.colorbar()
    cb.set_label('Power [dB re to {} {}^2]'.format(ref_value, raw_unit), rotation=270, labelpad=+15)
    
    
    plt.xlabel('time(samples)')
    plt.ylabel('dBa relative to 1 uPa**2/Hz')
    plt.axis([(1.450/3)*plt.xlim()[1], (1.650/3)*plt.xlim()[1], (16000/50000)*plt.ylim()[1], (20000/50000)*plt.ylim()[1]])
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'raw_unit' is not defined

In [61]:
y, sr=open_wav(the_wav, sample_rate)

plt.xlim()[1]

48.00512

In [ ]:
    
    nfft=1024
    hop_length=32
    win_length=400